In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [10]:
customers = pd.read_csv(r'D:\Customer Spend Risk Analysis\Data\Bronze\customers__bronze.csv')
merchants = pd.read_csv(r'D:\Customer Spend Risk Analysis\Data\Bronze\merchants_Bronze.csv')
transactions = pd.read_csv(r'D:\Customer Spend Risk Analysis\Data\Bronze\transactions_bronze.csv')

In [3]:
# SILVER - Transactions 

transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   txn_id       500 non-null    object 
 1   txn_date     500 non-null    object 
 2   customer_id  500 non-null    object 
 3   amount       249 non-null    float64
 4   currency     500 non-null    object 
 5   merchant     500 non-null    object 
 6   city         348 non-null    object 
 7   device       290 non-null    object 
dtypes: float64(1), object(7)
memory usage: 31.4+ KB


In [4]:
transactions.head()

,txn_id,txn_date,customer_id,amount,currency,merchant,city,device
0,TXN0001,22-01-2024,C036,NaN,INR,Unknown,Mumbai,NaN
1,TXN0002,2024/01/02,C070,NaN,usd,Zomato,Mumbai,Mobile
2,TXN0003,2024-02-18,C032,47261.0,usd,Flipkart,Delhi,NaN
3,TXN0004,2024-02-12,C073,NaN,INR,Unknown,Bangalore,Desktop
4,TXN0005,2024/02/08,C096,183313.0,usd,Walmart,Mumbai,NaN


In [11]:
#FIXING DATE TIME FORMAT

transactions['txn_date'].head()

0    22-01-2024
1    2024/01/02
2    2024-02-18
3    2024-02-12
4    2024/02/08
Name: txn_date, dtype: object

In [12]:
transactions['txn_date'] = (transactions['txn_date'].astype(str)
                            .str.replace('/', '-' , regex=False))

In [15]:
transactions['txn_date'].head()

0    22-01-2024
1    2024-01-02
2    2024-02-18
3    2024-02-12
4    2024-02-08
Name: txn_date, dtype: object

In [16]:
transactions['txn_date'] = pd.to_datetime(transactions['txn_date'],
                                          format='%Y-%m-%d',
                                          errors='coerce')

In [17]:
transactions['txn_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 500 entries, 0 to 499
Series name: txn_date
Non-Null Count  Dtype         
--------------  -----         
324 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 4.0 KB


In [18]:
# AMOUNT CLEANING

transactions['amount'].head()

0         NaN
1         NaN
2     47261.0
3         NaN
4    183313.0
Name: amount, dtype: float64

In [19]:
transactions['amount'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 500 entries, 0 to 499
Series name: amount
Non-Null Count  Dtype  
--------------  -----  
249 non-null    float64
dtypes: float64(1)
memory usage: 4.0 KB


In [20]:
transactions['amount'].fillna(0, inplace=True)

In [23]:
#CURRENCY STANDARDIZATIONS

transactions['currency'].head()

0    INR 
1     usd
2     usd
3    INR 
4     usd
Name: currency, dtype: object

In [25]:
transactions['currency'] = (transactions['currency'].str.strip().str.upper())
transactions['currency'].head()

0    INR
1    USD
2    USD
3    INR
4    USD
Name: currency, dtype: object

In [26]:
transactions['currency'].isnull().sum()

np.int64(0)

In [27]:
#CITY AND DEVICE CLEANING
transactions['city'].head()

0       Mumbai
1       Mumbai
2        Delhi
3    Bangalore
4       Mumbai
Name: city, dtype: object

In [28]:
transactions['city'].isnull().sum()

np.int64(152)

In [29]:
transactions['city'].fillna('Unknown', inplace=True)
transactions['device'].fillna('Unknown', inplace=True)
transactions['merchant'].fillna('Unknown', inplace=True)

In [30]:
transactions.head()

,txn_id,txn_date,customer_id,amount,currency,merchant,city,device
0,TXN0001,NaT,C036,0.0,INR,Unknown,Mumbai,Unknown
1,TXN0002,2024-01-02,C070,0.0,USD,Zomato,Mumbai,Mobile
2,TXN0003,2024-02-18,C032,47261.0,USD,Flipkart,Delhi,Unknown
3,TXN0004,2024-02-12,C073,0.0,INR,Unknown,Bangalore,Desktop
4,TXN0005,2024-02-08,C096,183313.0,USD,Walmart,Mumbai,Unknown


In [31]:
transactions['high_value_flag'] = transactions['amount'].apply(
    lambda x: 1 if x > 1000 else 0
)

In [32]:
silver_transactions = transactions.copy()

In [33]:
# SILVER - Customers

customers.head()

,customer_id,name,age,annual_income,segment,city
0,C001,Customer_1,58.0,NaN,VIP,Delhi
1,C002,Customer_2,32.0,NaN,PREMIUM,NaN
2,C003,Customer_3,NaN,1084834,Premium,Mumbai
3,C004,Customer_4,31.0,NaN,Premium,NaN
4,C005,Customer_5,NaN,not available,Regular,New York


In [34]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    120 non-null    object 
 1   name           120 non-null    object 
 2   age            36 non-null     float64
 3   annual_income  76 non-null     object 
 4   segment        120 non-null    object 
 5   city           76 non-null     object 
dtypes: float64(1), object(5)
memory usage: 5.8+ KB


In [35]:
customers['age'] = pd.to_numeric(customers['age'] , errors='coerce')
customers['annual_income'] = pd.to_numeric(customers['annual_income'] , errors='coerce')

In [36]:
customers['age'].isnull().sum()

np.int64(84)

In [37]:
customers['age'].fillna(customers['age'].median(), inplace=True)

In [38]:
customers['age'].isnull().sum()

np.int64(0)

In [39]:
customers['age'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 120 entries, 0 to 119
Series name: age
Non-Null Count  Dtype  
--------------  -----  
120 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


In [40]:
customers['annual_income'].isnull().sum()

np.int64(85)

In [42]:
customers['annual_income'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 120 entries, 0 to 119
Series name: annual_income
Non-Null Count  Dtype  
--------------  -----  
35 non-null     float64
dtypes: float64(1)
memory usage: 1.1 KB


In [43]:
customers['annual_income'].fillna(customers['annual_income'].median(), inplace=True)
customers['annual_income'].isnull().sum()

np.int64(0)

In [44]:
customers['segment'].head()

0        VIP
1    PREMIUM
2    Premium
3    Premium
4    Regular
Name: segment, dtype: object

In [45]:
customers['segment'].str.strip().str.upper()

0          VIP
1      PREMIUM
2      PREMIUM
3      PREMIUM
4      REGULAR
        ...   
115    PREMIUM
116    PREMIUM
117    REGULAR
118        VIP
119    REGULAR
Name: segment, Length: 120, dtype: object

In [46]:
customers.isnull().sum()

customer_id       0
name              0
age               0
annual_income     0
segment           0
city             44
dtype: int64

In [47]:
customers['city'].fillna("Unknown", inplace=True)

In [55]:
silver_customers = customers.copy()

In [48]:
# SILVERS  - MERCHANTS

merchants.head()

,merchant,category,risk_score
0,Amazon,NaN,0.12
1,Flipkart,NaN,0.57
2,Walmart,Ecommerce,0.17
3,Myntra,NaN,0.21
4,Swiggy,NaN,0.25


In [49]:
merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   merchant    7 non-null      object 
 1   category    3 non-null      object 
 2   risk_score  7 non-null      float64
dtypes: float64(1), object(2)
memory usage: 300.0+ bytes


In [50]:
merchants.isnull().sum()    

merchant      0
category      4
risk_score    0
dtype: int64

In [51]:
merchants['category'].fillna("Unknown", inplace=True)


In [52]:
merchants['risk_score'] = pd.to_numeric(merchants['risk_score'], errors='coerce')

In [53]:
merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   merchant    7 non-null      object 
 1   category    7 non-null      object 
 2   risk_score  7 non-null      float64
dtypes: float64(1), object(2)
memory usage: 300.0+ bytes


In [54]:
silver_merchants = merchants.copy()

In [57]:
import os

# SILVER SAVE FILES
os.makedirs("data/silver", exist_ok=True)

silver_transactions.to_csv(
    "data/silver/silver_transactions_clean.csv", index=False
)
silver_customers.to_csv(
    "data/silver/silver_customers_clean.csv", index=False
)
silver_merchants.to_csv(
    "data/silver/silver_merchants_clean.csv", index=False
)

print("Silver layer created successfully")

Silver layer created successfully
